In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import pandas as pd
import os
import time
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
from pathlib import Path

In [ ]:
base_url = 'https://whoicf2.whoi.edu/science/B/whalesounds/metaData.cfm?RN='

ds_path = 'dataset'
if not os.path.exists(ds_path):
    os.makedirs(ds_path)
    
# Campi dei metadati da estrarre
col = ['filename', 'CU:', 'SR:', 'CS:']
metadata_df = pd.DataFrame(columns=col)

In [ ]:
#  Funzione per l'estrazione dei metadati
def retrieve_metadata(folder_path, base_url):
    ret_list = []
    for f in os.listdir(folder_path):
        fn = f.split('.')[0]
        curr_dict = {}
        curr_url = base_url + fn
        curr_dict['filename'] = fn
        curr_dict['species'] = f

        # Richiesta html e parsing della tabella
        html = requests.get(curr_url)
        soup = BeautifulSoup(html.content, 'html.parser')
        rows = soup.find_all('tr')

        # Estrazione dei dati dalla tabella
        campi_desiderati = col[1:]
        for row in rows:
            celle = row.find_all('td')
            if len(celle) == 2:
                chiave = celle[0].get_text(strip=True)
                valore = celle[1].get_text(strip=True)
                if chiave in campi_desiderati:
                    curr_dict[chiave] = valore
        ret_list.append(curr_dict)

    return ret_list


In [ ]:
# Salvataggio dei metadati di tutte le specie in un unico file csv
for folder in os.listdir('dataset'):
    folder_dict = retrieve_metadata(Path(ds_path, folder), base_url)
    metadata_df = pd.concat([metadata_df, pd.DataFrame.from_records(folder_dict)], ignore_index=True)
metadata_df.to_csv('metadata.csv', index=False)
